In [1]:
import os
import cv2
import time
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt

from keras.models import model_from_json
#from utils_gen import gen_paths_img_dm, gen_var_from_paths
#from utils_imgproc import norm_by_imagenet
%matplotlib inline
plt.ioff()
import glob




import cv2
import numpy as np


def smallize_density_map(density_map, stride=1):
    if stride > 1:
        density_map_stride = np.zeros((np.asarray(density_map.shape).astype(int)//stride).tolist(), dtype=np.float32)
        for r in range(density_map_stride.shape[0]):
            for c in range(density_map_stride.shape[1]):
                density_map_stride[r, c] = np.sum(density_map[r*stride:(r+1)*stride, c*stride:(c+1)*stride])
    else:
        density_map_stride = density_map
    return density_map_stride


def norm_by_imagenet(img):
    if len(img.shape) == 3:
        img = img / 255.0
        img[:, :, 0] = (img[:, :, 0] - 0.485) / 0.229
        img[:, :, 1] = (img[:, :, 1] - 0.456) / 0.224
        img[:, :, 2] = (img[:, :, 2] - 0.406) / 0.225
        return img
    elif len(img.shape) == 4 or len(img.shape) == 1:
        # In SHA, shape of images varies, so the array.shape is (N, ), that's the '== 1' case.
        imgs = []
        for im in img:
            im = im / 255.0
            im[:, :, 0] = (im[:, :, 0] - 0.485) / 0.229
            im[:, :, 1] = (im[:, :, 1] - 0.456) / 0.224
            im[:, :, 2] = (im[:, :, 2] - 0.406) / 0.225
            imgs.append(im)
        return np.array(imgs)
    else:
        print('Wrong shape of the input.')
        return None



def image_preprocessing(x, y, flip_hor=False, brightness_shift=False):
    xs, ys = [], []
    for idx_pro in range(x.shape[0]):
        x_, y_ = x[idx_pro], y[idx_pro]
        # preprocessings -----
        if flip_hor:
            x_, y_ = flip_horizontally(x_, y_)
        # preprocessings -----
        x_ = norm_by_imagenet(x_)
        #print(x_)
        #print(x_.shape)
        #print(y_)
        #print(y_.shape)
        xs.append(x_)
        ys.append(y_)
    xs, ys = np.array(xs), np.array(ys)
    return xs, ys



def flip_horizontally(x, y):
    to_flip = np.random.randint(0, 2)
    if to_flip:
        x, y = cv2.flip(x, 1), np.expand_dims(cv2.flip(np.squeeze(y), 1), axis=-1)
        # Suppose shape of y is (123, 456, 1), after cv2.flip, shape of y would turn into (123, 456).
    return x, y


def fix_singular_shape(img, unit_len=16):
    """
    Some network like w-net has both N maxpooling layers and concatenate layers,
    so if no fix for their shape as integeral times of 2 ** N, the shape will go into conflict.
    """
    #hei_dst, wid_dst = img.shape[0] + (unit_len - img.shape[0] % unit_len), img.shape[1] + (unit_len - img.shape[1] % unit_len)
    hei_dst,wid_dst=900,900
    if len(img.shape) == 3:
        img = cv2.resize(img, (hei_dst, wid_dst), interpolation=cv2.INTER_LANCZOS4)#i changes it to 900 and same down
    elif len(img.shape) == 2:
        GT = int(round(np.sum(img)))
        img = cv2.resize(img, (hei_dst, wid_dst), interpolation=cv2.INTER_LANCZOS4)
        img = img / (np.sum(img) / GT)
    return img

import os
import cv2
import h5py
import scipy
import numpy as np

from keras.preprocessing.image import ImageDataGenerator
#from utils_imgproc import smallize_density_map, fix_singular_shape


def gen_paths_img_dm(path_inp):
    img_paths = []
    dm_paths=[]
    ips = os.path.join(path_inp, 'images')
    dps= os.path.join(path_inp, 'ground-truth-h5')
    ipaths=[ips]
    dpaths=[dps]

    for path in ipaths:
    
        for img_path in glob.glob(os.path.join(path, '*.jpg')):
        
           img_paths.append(str(img_path))
        
    print("Total images : ",len(img_paths))

    for path in dpaths:
    
        for dm_path in glob.glob(os.path.join(path, '*.h5')):
        
             dm_paths.append(str(dm_path))
        
     #print("Total images : ",len(img_paths))

    return img_paths, dm_paths

def gen_var_from_paths(paths, stride=1, unit_len=16):
    vars = []
    unit_len=10
    format_suffix = paths[0].split('.')[-1]
    if format_suffix == 'h5':
        for ph in paths:
            print(h5py.File(ph, 'r')['density'])
            dm = h5py.File(ph, 'r')['density'][()].astype(np.float32)
            print(dm)
            #if unit_len:
            dm = fix_singular_shape(dm, unit_len=unit_len)
            print(dm)
            dm = smallize_density_map(dm, stride=stride)
            print(dm)
            vars.append(np.expand_dims(dm, axis=-1))
    elif format_suffix == 'jpg':
        for ph in paths:
            raw = cv2.cvtColor(cv2.imread(ph), cv2.COLOR_BGR2RGB).astype(np.float32)
            #if unit_len:
            raw = fix_singular_shape(raw, unit_len=unit_len)
            vars.append(raw)
        # vars = norm_by_imagenet(vars)
    else:
        print('Format suffix is wrong.')
    return np.array(vars)

def gen_density_map_gaussian(im, points, sigma=4):
    """
    func: generate the density map
    """
    density_map = np.zeros(im.shape[:2], dtype=np.float32)
    h, w = density_map.shape[:2]
    num_gt = np.squeeze(points).shape[0]
    if num_gt == 0:
        return density_map
    if sigma == 4:
        # Adaptive sigma in CSRNet.
        leafsize = 2048
        tree = scipy.spatial.KDTree(points.copy(), leafsize=leafsize)
        distances, _ = tree.query(points, k=4)
    for idx_p, p in enumerate(points):
        p = np.round(p).astype(int)
        p[0], p[1] = min(h-1, p[1]), min(w-1, p[0])
        gaussian_radius = sigma * 2 - 1
        if sigma == 4:
            # Adaptive sigma in CSRNet.
            sigma = max(int(np.sum(distances[idx_p][1:4]) * 0.1), 1)
            gaussian_radius = sigma * 3
        gaussian_map = np.multiply(
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma),
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma).T
        )
        x_left, x_right, y_up, y_down = 0, gaussian_map.shape[1], 0, gaussian_map.shape[0]
        # cut the gaussian kernel
        if p[1] < gaussian_radius:
            x_left = gaussian_radius - p[1]
        if p[0] < gaussian_radius:
            y_up = gaussian_radius - p[0]
        if p[1] + gaussian_radius >= w:
            x_right = gaussian_map.shape[1] - (gaussian_radius + p[1] - w) - 1
        if p[0] + gaussian_radius >= h:
            y_down = gaussian_map.shape[0] - (gaussian_radius + p[0] - h) - 1
        gaussian_map = gaussian_map[y_up:y_down, x_left:x_right]
        if np.sum(gaussian_map):
            gaussian_map = gaussian_map / np.sum(gaussian_map)
        density_map[
            max(0, p[0]-gaussian_radius):min(h, p[0]+gaussian_radius+1),
            max(0, p[1]-gaussian_radius):min(w, p[1]+gaussian_radius+1)
        ] += gaussian_map
    density_map = density_map / (np.sum(density_map / num_gt))
    return density_map


import os
import cv2
import h5py
import scipy
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
#from utils_imgproc import smallize_density_map, fix_singular_shape
import keras.backend as K


def MSE_BCE(y_true, y_pred, alpha=1000, beta=10):
    mse = K.mean(K.square(y_true - y_pred), axis=-1)
    bce = K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    return alpha * mse + beta * bce

    

import os
import cv2
import numpy as np
#from skimage.measure import compare_psnr, compare_ssim
import matplotlib.pyplot as plt


def callbacks_during_train(model, dis_x, dis_y, dis_path, net, epoch):
    if not os.path.exists('tmp'):
        os.mkdir('tmp')
    # show prediction
    pred = np.squeeze(model.predict(np.expand_dims(dis_x, axis=0)))
    _, (ax_x_ori, ax_y, ax_pred) = plt.subplots(1, 3, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(dis_path), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    ax_y.imshow(np.squeeze(dis_y), cmap=plt.cm.jet)
    ax_y.set_title('Ground_truth: ' + str(np.sum(dis_y)))
    ax_pred.imshow(pred, cmap=plt.cm.jet)
    ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.savefig('tmp/{}_{}-epoch.png'.format(net, epoch))
    return None


def eval_loss(model, x, y, quality=False):
    preds, DM, GT = [], [], []
    losses_SFN, losses_MAE, losses_MAPE, losses_RMSE = [], [], [], []
    for idx_pd in range(x.shape[0]):
        pred = model.predict(np.array([x[idx_pd]]))
        preds.append(np.squeeze(pred))
        DM.append(np.squeeze(np.array([y[idx_pd]])))
        GT.append(round(np.sum(np.array([y[idx_pd]]))))    # To make sure the GT is an integral value
    for idx_pd in range(len(preds)):
        losses_SFN.append(np.mean(np.square(preds[idx_pd] - DM[idx_pd])))     # mean of Frobenius norm
        losses_MAE.append(np.abs(np.sum(preds[idx_pd]) - GT[idx_pd]))
        losses_MAPE.append(np.abs(np.sum(preds[idx_pd]) - GT[idx_pd]) / GT[idx_pd])
        losses_RMSE.append(np.square(np.sum(preds[idx_pd]) - GT[idx_pd]))

    loss_SFN = np.sum(losses_SFN)
    loss_MAE = np.mean(losses_MAE)
    loss_MAPE = np.mean(losses_MAPE)
    loss_RMSE = np.sqrt(np.mean(losses_RMSE))
    if quality:

        psnr, ssim = [], []
        for idx_pd in range(len(preds)):
            data_range = max([np.max(preds[idx_pd]), np.max(DM[idx_pd])]) - min([np.min(preds[idx_pd]), np.min(DM[idx_pd])])
            psnr_ = compare_psnr(preds[idx_pd], DM[idx_pd], data_range=data_range)
            ssim_ = compare_ssim(preds[idx_pd], DM[idx_pd], data_range=data_range)
            psnr.append(psnr_)
            ssim.append(ssim_)
        return loss_MAE, loss_RMSE, loss_SFN, loss_MAPE, np.mean(psnr), np.mean(ssim)
    print("MAE="+str(loss_MAE))
    print("RMSE="+str(loss_RMSE))
    return loss_MAE, loss_RMSE, loss_SFN, loss_MAPE

net = 'Crowd'
dataset = "B"

In [2]:
import tensorflow as tf
print(tf.__version__)

2.6.2


In [ ]:
(train_img_paths,train_dm_paths)=gen_paths_img_dm('/content/drive/MyDrive/ShanghaiTech/part_B/train_data')
train_x, train_y = gen_var_from_paths(train_img_paths[:], unit_len=None), gen_var_from_paths(train_dm_paths[:], stride=8, unit_len=None)
print('Train data size:', train_x.shape[0], train_y.shape[0], len(train_img_paths))


In [ ]:
train_x.shape
train_y.shape
import gc
gc.collect()
(test_img_paths,test_dm_paths)=gen_paths_img_dm('/content/drive/MyDrive/ShanghaiTech/part_B')
test_x, test_y = gen_var_from_paths(test_img_paths[:], unit_len=None), gen_var_from_paths(test_dm_paths[:], stride=8, unit_len=None)
test_x = norm_by_imagenet(test_x)  # Normalization on raw images in test set, those of training set are in image_preprocessing below.
print('Test data size:', test_x.shape[0], test_y.shape[0], len(test_img_paths))

In [3]:
import numpy as np
train_x=np.load('../input/federatedlearning/TrainSTBXIP.npy')
train_y=np.load('../input/federatedlearning/TrainSTBYIP.npy')
test_x=np.load('../input/federatedlearning/TestSTBx.npy')
test_y=np.load('../input/federatedlearning/TestSTBy.npy')

In [4]:
length = len(test_x)
middle_index = 100
testx = test_x[:middle_index]
length = len(test_y)
middle_index = 100
testy = test_y[:middle_index]

In [5]:
length = len(train_x)
middle_index = length//2
#Floor division rounds down


client1x = train_x[:middle_index]

client2x = train_x[middle_index:]

In [6]:
length=len(client1x)
mid = length//2
c1x = client1x[ :mid]
c2x=client1x[mid:]

In [7]:
length=len(client2x)
mid = length//2
c3x = client2x[ :mid]
c4x=client2x[mid:]

In [8]:
length = len(train_y)
middle_index = length//2

client1y = train_y[:middle_index]

client2y = train_y[middle_index:]

In [9]:
length=len(client1y)
mid = length//2
c1y = client1y[ :mid]
c2y=client1y[mid:]

In [10]:
length=len(client2y)
mid = length//2
c3y = client2y[ :mid]
c4y=client2y[mid:]

In [11]:
c4y.shape

(100, 128, 96, 1)

In [12]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Conv2D, Input, MaxPooling2D, concatenate, add, UpSampling2D
from keras.models import Model
from keras.initializers import RandomNormal


from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.layers.wrappers import TimeDistributed

def stbflmodel(input_shape=(None, None, 3)):

    input_flow = Input(shape=input_shape)
    dilated_conv_kernel_initializer = RandomNormal(stddev=0.01)

    # front-end
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_flow)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)

    # back-end
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(512, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(256, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', dilation_rate=2, activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)

    output_flow = Conv2D(1, 1, strides=(1, 1), padding='same', activation='relu', kernel_initializer=dilated_conv_kernel_initializer)(x)
    model = Model(inputs=input_flow, outputs=output_flow)

    front_end = VGG16(weights='imagenet', include_top=False)

    weights_front_end = []
    for layer in front_end.layers:
        if 'conv' in layer.name:
            weights_front_end.append(layer.get_weights())
    counter_conv = 0
    for i in range(len(front_end.layers)):
        if counter_conv >= 10:
            break
        if 'conv' in model.layers[i].name:
            model.layers[i].set_weights(weights_front_end[counter_conv])
            counter_conv += 1

    return model

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
LOSS = 'MSE'
optimizer = Adam(learning_rate=1e-5)
model = stbflmodel(input_shape=(None,None, 3))
LOSS = 'MSE'
metrics = ['accuracy']
#optimizer = Adam(learning_rate=1e-5)
#model = stbflmodel(input_shape=(None,None, 3))

#model.compile(optimizer=optimizer, loss='MSE')
model.compile(optimizer=optimizer, loss='MSE',metrics=metrics)
model.summary()
if not os.path.exists('models'):
    os.makedirs('models')
plot_model(model, 'models/{}.png'.format(net))
with open('./models/{}.json'.format(net), 'w') as fout:
    fout.write(model.to_json())


2022-01-30 19:44:17.564416: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 19:44:17.667562: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 19:44:17.668249: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-30 19:44:17.670196: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 128)   73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 128)   147584    
______________________________________________________________

In [13]:
def weight_scalling_factor():
    
    global_count = 400
    
    local_count = 200
    
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [14]:
def weight_scaling_factor():
    
    global_count = 400
    
    local_count = 100
    
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [15]:
import gc
gc.collect()
#train_y.shape

6859

In [ ]:


LOSS = 'MSE'
metrics = ['accuracy']
optimizer = Adam(learning_rate=1e-5)
global_model = stbflmodel(input_shape=(None,None, 3))

global_model.compile(optimizer=optimizer, loss='MSE',metrics=metrics)
#new_model = tf.keras.models.load_model('my_model.h5')

In [16]:
LOSS = 'MSE'
metrics = ['accuracy']
optimizer = Adam(learning_rate=1e-5)
global_model = stbflmodel(input_shape=(None,None, 3))

global_model.compile(optimizer=optimizer, loss='MSE')

In [ ]:
import numpy as np
#train_x=np.load('/content/drive/MyDrive/TrainSTBXIP.npy')
#train_y=np.load('/content/drive/MyDrive/TrainSTBYIP.npy')
testx=np.load('/content/drive/MyDrive/TestSTBx.npy')
testy=np.load('/content/drive/MyDrive/TestSTBy.npy')
gl= eval_loss(global_model, testx, testy, quality=False)

In [17]:
for i in range(100):
     print("comm_round"+str(i))
     global_weights = global_model.get_weights()
     scaled_local_weight_list = list()

     print("enter client1")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model = stbflmodel(input_shape=(None,None, 3))

     local_model.compile(optimizer=optimizer, loss='MSE')
        
     local_model.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 100, 16):
        
           x, y = c1x[idx_train:idx_train+16], c1y[idx_train:idx_train+16]
       
           local_model.fit(x, y, batch_size=1, verbose=1)
        
       
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)        
     print("enter client2")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model2 = stbflmodel(input_shape=(None,None, 3))

     local_model2.compile(optimizer=optimizer, loss='MSE')
        #set local model weight to the weight of the global model
     local_model2.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 100, 16):
        
           x, y = c2x[idx_train:idx_train+16], c2y[idx_train:idx_train+16]
    
           local_model2.fit(x, y, batch_size=1, verbose=1)
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model2.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)  
     #average_weights = sum_scaled_weights(scaled_local_weight_list)
     
     print("enter client3")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model3 = stbflmodel(input_shape=(None,None, 3))

     local_model3.compile(optimizer=optimizer, loss='MSE')
        
     local_model3.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 100, 16):
        
           x, y = c3x[idx_train:idx_train+16], c3y[idx_train:idx_train+16]
       
           local_model3.fit(x, y, batch_size=1, verbose=1)
        
       
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model3.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)
     
    
     print("enter client4")
        
     LOSS = 'MSE'
     metrics = ['accuracy']
     optimizer = Adam(learning_rate=1e-5)
     local_model4 = stbflmodel(input_shape=(None,None, 3))

     local_model4.compile(optimizer=optimizer, loss='MSE')
        
     local_model4.set_weights(global_weights)
     for ep in range(1):
        for idx_train in range(0, 100, 16):
        
           x, y = c4x[idx_train:idx_train+16], c4y[idx_train:idx_train+16]
       
           local_model4.fit(x, y, batch_size=1, verbose=1)
        
       
        
     scaling_factor = weight_scaling_factor()
     scaled_weights = scale_model_weights(local_model4.get_weights(), scaling_factor)
     scaled_local_weight_list.append(scaled_weights)
     average_weights = sum_scaled_weights(scaled_local_weight_list)
    #update global model 
     global_model.set_weights(average_weights)
     print("global test")
     
     gl= eval_loss(global_model, test_x, test_y, quality=False)

comm_round0
enter client1


2022-01-30 19:44:28.984148: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-30 19:44:29.188288: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-30 19:44:29.320506: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-30 19:44:31.063915: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


16/16 [==============================] - 13s 170ms/step - loss: 0.0025


2022-01-30 19:44:41.947390: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.
2022-01-30 19:44:42.159750: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.


16/16 [==============================] - 3s 171ms/step - loss: 0.0024


2022-01-30 19:44:45.067084: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 152275200 exceeds 10% of free system memory.


4/4 [==============================] - 1s 168ms/step - loss: 0.0032
enter client2
4/4 [==============================] - 1s 168ms/step - loss: 4.1067e-05
enter client3
4/4 [==============================] - 1s 168ms/step - loss: 0.0011
enter client4
4/4 [==============================] - 1s 168ms/step - loss: 0.0075
global test
MAE=62.92890473558933
RMSE=85.94873938720038
comm_round1
enter client1
4/4 [==============================] - 1s 168ms/step - loss: 0.0028
enter client2
4/4 [==============================] - 1s 168ms/step - loss: 2.8745e-05
enter client3
4/4 [==============================] - 1s 168ms/step - loss: 8.8741e-04
enter client4
4/4 [==============================] - 1s 167ms/step - loss: 0.0069
global test
MAE=30.75083244299587
RMSE=50.53476013637212
comm_round2
enter client1
4/4 [==============================] - 1s 168ms/step - loss: 0.0026
enter client2
4/4 [==============================] - 1s 168ms/step - loss: 2.4957e-05
enter client3
4/4 [=====================

In [ ]:

testx=np.load('/content/drive/MyDrive/TestSTBx.npy')
testy=np.load('/content/drive/MyDrive/TestSTBy.npy')

In [18]:
global_model.save('fedlmodel4c100.h5')

In [ ]:
import numpy as np
import tensorflow as tf
testx=np.load('../input/federatedlearning/TestSTBx.npy')
testy=np.load('../input/federatedlearning/TestSTBy.npy')
globalmodel = tf.keras.models.load_model('./fedlmodel4c.h5')
gl= eval_loss(globalmodel, testx, testy, quality=False)

In [ ]:
import numpy as np
import tensorflow as tf
testx=np.load('../input/federatedlearning/TestSTBx.npy')
testy=np.load('../input/federatedlearning/TestSTBy.npy')
globalmodel = tf.keras.models.load_model('../input/weights/fedlmodel.h5')
gl= eval_loss(globalmodel, testx, testy, quality=False)

In [19]:
globalmodel = tf.keras.models.load_model('./fedlmodel4c100.h5')
gl= eval_loss(globalmodel, test_x, test_y, quality=False)

MAE=14.241957821423494
RMSE=21.179114630959635


In [21]:
test_x.shape

(316, 1030, 770, 3)

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
                          

In [ ]:
#gl= eval_loss(globalmodel, testx, testy, quality=False)